In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
import cv2
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import v2
import timm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
"""
#label_csv is like this:
#
# image_id, probabilty_Si, probabilty_SWCNT

"""
class kanaiDataset(Dataset):
    def __init__(self, image_data_path, label_csv_path, transform=None):
        self.transform = transform
        self.label_df = pd.read_csv(label_csv_path, comment='#', header=None, sep=None, engine='python', encoding='cp932')

        self.images = []
        self.labels = []
        for i in range(len(self.label_df)):
            if not os.path.exists(os.path.join(image_data_path, str(self.label_df.iloc[i, 0])+'.tiff')):
                print(f"Image {str(self.label_df.iloc[i, 0])}.tiff not found in {image_data_path}")
                continue
            image = PIL.Image.open(os.path.join(image_data_path, str(self.label_df.iloc[i, 0])+'.tiff'))
            label = torch.tensor([self.label_df.iloc[i, 1], self.label_df.iloc[i, 2]])
            self.images.append(image)
            self.labels.append(label)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
transform = v2.Compose([
    v2.ToImage(), #torch Image型への変換。channel/height/widthの順番に変換される。
    v2.ToDtype(torch.float16, scale=True), #torch.float16型への変換。scale=Trueで0-1の範囲に変換される。
    #PLの測定データに関しては各ピクセルの値（各CCD素子の値）自体も重要な情報なので、scale=Falseにしても良いかもしれない。
])

In [ ]:
model = timm.create_model('tf_efficientnetv2_b2.in1k', pretrained=True)

In [ ]:
lr = 1e-4

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)